In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
from statsmodels.tsa.vector_ar.svar_model import SVAR,SVARResults
from statsmodels.tsa.arima.model import ARIMA
np.set_printoptions(suppress=True)

In [4]:
df = pd.read_excel('oil_and_gas_index.xlsx')
df = df.dropna()
df = df.resample('M', on='Dates').mean()
print(df.head())
df1 = pd.DataFrame()
df1.index = df.index
df1['Oil Return'] = np.log(df['CL1 Comdty']).diff().dropna()
df1['Oil_and_Gas_Producers'] = np.log(df['SPGOGUP Index']).diff().dropna()
model = ARIMA(df['VIX Index'],order=(1,0,1)).fit()
VIX_resid = model.resid
df1['VIX'] = VIX_resid
df1 = df1.dropna()
df1 = df1['2005-03-31':]
print(df1.head())

            CL1 Comdty  VIX Index  SPGOGUP Index
Dates                                           
2000-08-31   33.120000  16.840000    1000.000000
2000-09-30   33.843810  19.584762    1012.718571
2000-10-31   32.932273  25.200000     991.575000
2000-11-30   34.366364  26.443182     965.590000
2000-12-31   28.292857  26.579048     951.973333
            Oil Return  Oil_and_Gas_Producers       VIX
Dates                                                  
2005-03-31    0.128126               0.047681  0.256350
2005-04-30   -0.026353              -0.027281 -0.152651
2005-05-31   -0.063149              -0.030208 -1.696425
2005-06-30    0.121575               0.070683 -2.988402
2005-07-31    0.044940               0.054450 -1.972985


In [5]:
A = np.asarray([[1,1,1],[0,'E','E'],[0,0,'E']])
svar_model = SVAR(df1,svar_type='A',A=A)
res = svar_model.fit(maxlags=3, maxiter=10000, maxfun=10000, solver='bfgs')
resid = res.resid

dates = df.index[4:]
print(dates)
supply_shock = []
demand_shock = []
market_shock = []
for i in range(len(resid)):
    supply_shock.append(resid[i][0])
    demand_shock.append(resid[i][1])
    market_shock.append(resid[i][2])

pio.renderers.default = "browser"

fig = go.Figure(go.Scatter(x=dates,y=supply_shock))
fig.show()

c:\Users\saeed\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


DatetimeIndex(['2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31',
               '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31',
               '2001-08-31', '2001-09-30',
               ...
               '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
               '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30',
               '2023-07-31', '2023-08-31'],
              dtype='datetime64[ns]', name='Dates', length=273, freq='M')


In [6]:
fig = go.Figure(go.Scatter(x=dates,y=demand_shock))
fig.show()

In [7]:
fig = go.Figure(go.Scatter(x=dates,y=market_shock))
fig.show()

In [8]:
shocks = np.array([supply_shock,demand_shock,market_shock])
corr = np.corrcoef(shocks)
corr

array([[ 1.        ,  0.66529889, -0.37441816],
       [ 0.66529889,  1.        , -0.7430562 ],
       [-0.37441816, -0.7430562 ,  1.        ]])

In [9]:
df1.tail()


,Oil Return,Oil_and_Gas_Producers,VIX
Dates,,,
2023-04-30,0.080225,0.043207,-3.702612
2023-05-31,-0.103752,-0.059271,0.005067
2023-06-30,-0.018644,0.009124,-4.113150
2023-07-31,0.073889,0.039637,-0.593096
2023-08-31,0.069523,0.032290,0.879804


In [10]:
import statsmodels.api as sm
print(df1.head())
mod = sm.OLS( df1['Oil_and_Gas_Producers'],df1['VIX']).fit()
#model_reg.fit(np.array(df1['VIX']).reshape(-1,1),np.array(df1['Oil_and_Gas_Producers']).reshape(-1,1))
print(mod.summary())
fig = go.Figure(go.Scatter(x=dates,y=mod.resid))
fig.show()

df1['Demand_Shock'] = mod.resid


            Oil Return  Oil_and_Gas_Producers       VIX
Dates                                                  
2005-03-31    0.128126               0.047681  0.256350
2005-04-30   -0.026353              -0.027281 -0.152651
2005-05-31   -0.063149              -0.030208 -1.696425
2005-06-30    0.121575               0.070683 -2.988402
2005-07-31    0.044940               0.054450 -1.972985
                                  OLS Regression Results                                  
Dep. Variable:     Oil_and_Gas_Producers   R-squared (uncentered):                   0.498
Model:                               OLS   Adj. R-squared (uncentered):              0.496
Method:                    Least Squares   F-statistic:                              219.2
Date:                   Wed, 01 May 2024   Prob (F-statistic):                    6.44e-35
Time:                           09:17:49   Log-Likelihood:                          374.66
No. Observations:                    222   AIC:               

In [15]:
mod1 = sm.OLS( df1['Oil Return'],df1[['VIX','Demand_Shock']]).fit()
#model_reg.fit(np.array(df1['VIX']).reshape(-1,1),np.array(df1['Oil_and_Gas_Producers']).reshape(-1,1))
#print(mod1.summary())

print(mod1.summary())
df1['Supply_Shock'] = mod1.resid
#print(np.corrcoef([df['Oil_and_Gas_Producers']]))
print(np.round(np.corrcoef([df1['Oil Return'],mod1.resid,mod.resid,df1['VIX']]),4))
fig = go.Figure(go.Scatter(x=dates,y=mod1.resid))
fig.show()
df1.head()

                                 OLS Regression Results                                
Dep. Variable:             Oil Return   R-squared (uncentered):                   0.482
Model:                            OLS   Adj. R-squared (uncentered):              0.477
Method:                 Least Squares   F-statistic:                              102.3
Date:                Wed, 01 May 2024   Prob (F-statistic):                    3.91e-32
Time:                        09:23:30   Log-Likelihood:                          251.57
No. Observations:                 222   AIC:                                     -499.1
Df Residuals:                     220   BIC:                                     -492.3
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

,Oil Return,Oil_and_Gas_Producers,VIX,Demand_Shock,Supply_Shock
Dates,,,,,
2005-03-31,0.128126,0.047681,0.256350,0.050138,0.054619
2005-04-30,-0.026353,-0.027281,-0.152651,-0.028744,0.015748
2005-05-31,-0.063149,-0.030208,-1.696425,-0.046464,-0.005530
2005-06-30,0.121575,0.070683,-2.988402,0.042047,0.036872
2005-07-31,0.044940,0.054450,-1.972985,0.035543,-0.022680


In [12]:
df1.to_excel('Monthly_Results.xlsx')

In [21]:
dataset  = yf.download('^GSPC',start = '1998-12-30',end = '2023-08-30')
market_index = dataset['Adj Close']
market_return = np.log(market_index/market_index.shift(1)).dropna()
market_return.head()

[*********************100%***********************]  1 of 1 completed


Date
1998-12-31 00:00:00-05:00   -0.002194
1999-01-04 00:00:00-05:00   -0.000920
1999-01-05 00:00:00-05:00    0.013491
1999-01-06 00:00:00-05:00    0.021899
1999-01-07 00:00:00-05:00   -0.002053
Name: Adj Close, dtype: float64

In [22]:
df1.head()

,Oil Return,Oil_and_Gas_Producers,VIX,Demand_Shock,Supply_Shock
Dates,,,,,
2005-03-31,0.128126,0.047681,0.256350,0.050138,0.054619
2005-04-30,-0.026353,-0.027281,-0.152651,-0.028744,0.015748
2005-05-31,-0.063149,-0.030208,-1.696425,-0.046464,-0.005530
2005-06-30,0.121575,0.070683,-2.988402,0.042047,0.036872
2005-07-31,0.044940,0.054450,-1.972985,0.035543,-0.022680
